In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50, VGG19, DenseNet121, DenseNet201  # Use DenseNet201
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Data paths
train_data_dir = 'C://Users/HP/dataset_new_processed/train_template'
test_data_dir = 'C://Users/HP/dataset_new_processed/test_template/2'

# Parameters
input_shape = (224, 224, 3)
num_classes = 3  # Change this to the number of classes in your dataset
batch_size = 32
epochs = 10

# Load pre-trained models
resnet_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
vgg_model = VGG19(include_top=False, weights='imagenet', input_shape=input_shape)
densenet121_model = DenseNet121(include_top=False, weights='imagenet', input_shape=input_shape)
densenet201_model = DenseNet201(include_top=False, weights='imagenet', input_shape=input_shape)  # Use DenseNet201

# Fine-tune models
for layer in resnet_model.layers:
    layer.trainable = True

for layer in vgg_model.layers:
    layer.trainable = True

for layer in densenet121_model.layers:
    layer.trainable = True

for layer in densenet201_model.layers:
    layer.trainable = True

# Create inputs for each model
input_tensor = Input(shape=input_shape)
resnet_output = resnet_model(input_tensor)
vgg_output = vgg_model(input_tensor)
densenet121_output = densenet121_model(input_tensor)
densenet201_output = densenet201_model(input_tensor)

# Concatenate the outputs of all models
merged_output = tf.keras.layers.Concatenate()([resnet_output, vgg_output, densenet121_output, densenet201_output])

# Add global average pooling and final dense layers
x = tf.keras.layers.GlobalAveragePooling2D()(merged_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

# Create the ensemble model
ensemble_model = Model(inputs=input_tensor, outputs=output)

# Compile the ensemble model
ensemble_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
train_datagen = ImageDataGenerator(rescale=1.0/255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=input_shape[:2], batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_data_dir, target_size=input_shape[:2], batch_size=batch_size, class_mode='categorical')

# Train the ensemble model
ensemble_model.fit(train_generator, epochs=epochs, validation_data=test_generator)

# Evaluate the model
test_loss, test_acc = ensemble_model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

Found 1445 images belonging to 5 classes.
Found 394 images belonging to 5 classes.
Epoch 1/10
